Title: PyBlog - Jupyter Notebook to HTML Converter

Description:
PyBlog converts Jupyter Notebook files into beautifully styled HTML documents. With PyBlog, you can effortlessly transform your code, visualizations, and markdown content into a professional-looking blog post or documentation and store it on the cloud setting the permissions to decide if you want it to be shared on the public API.

Methods:

1. `convert_to_html(notebook_file: str, output_file: str) -> None`:
   - Description: This method converts a Jupyter Notebook file to HTML format.
   - Parameters:
     - `notebook_file` (str): The path to the Jupyter Notebook file to be converted.
     - `output_file` (str): The path to save the generated HTML file.
   - Returns: None

2. `apply_styling(html_file: str, css_file: str) -> None`:
   - Description: This method applies custom styling to the generated HTML file.
   - Parameters:
     - `html_file` (str): The path to the HTML file to be styled.
     - `css_file` (str): The path to the CSS file containing the styling rules.
   - Returns: None

3. `add_header(html_file: str, title: str, description: str) -> None`:
   - Description: This method adds a header section to the HTML file, including the title and description of the blog post or documentation.
   - Parameters:
     - `html_file` (str): The path to the HTML file to add the header section.
     - `title` (str): The title of the blog post or documentation.
     - `description` (str): A brief description of the content.
   - Returns: None

4. `add_code_highlighting(html_file: str) -> None`:
   - Description: This method adds syntax highlighting to code cells in the HTML file.
   - Parameters:
     - `html_file` (str): The path to the HTML file to add code highlighting.
   - Returns: None

5. `add_table_of_contents(html_file: str) -> None`:
   - Description: This method adds a table of contents to the HTML file, based on the notebook's headings and sections.
   - Parameters:
     - `html_file` (str): The path to the HTML file to add the table of contents.
   - Returns: None

In [1]:
%pip install -r requirements.txt

Defaulting to user installation because normal site-packages is not writeable
  Preparing metadata (setup.py) ... error
  error: subprocess-exited-with-error
  
  × python setup.py egg_info did not run successfully.
  │ exit code: 1
  ╰─> [70 lines of output]
      /usr/lib/python3/dist-packages/pkg_resources/__init__.py:116: PkgResourcesDeprecationWarning: 1.16.0-unknown is an invalid version and will not be supported in a future release
        warnings.warn(
      /usr/lib/python3/dist-packages/pkg_resources/__init__.py:116: PkgResourcesDeprecationWarning: 1.1build1 is an invalid version and will not be supported in a future release
        warnings.warn(
      /usr/lib/python3/dist-packages/pkg_resources/__init__.py:116: PkgResourcesDeprecationWarning: 0.1.43ubuntu1 is an invalid version and will not be supported in a future release
        warnings.warn(
      /usr/lib/python3/dist-packages/setuptools/installer.py:27: SetuptoolsDeprecationWarning: setuptools.installer is deprecate

In [2]:
from pymongo.mongo_client import MongoClient
from pymongo.server_api import ServerApi
# use environment variables to store the password
import os
from dotenv import load_dotenv
load_dotenv()

pw = os.environ.get("MONGO_PW")

uri = "mongodb+srv://twhornsby7:" + pw + "@pyblog.erwaxii.mongodb.net/?retryWrites=true&w=majority"

# Create a new client and connect to the server
client = MongoClient(uri, server_api=ServerApi('1'))

# Send a ping to confirm a successful connection
try:
    client.admin.command('ping')
    print("Pinged your deployment. You successfully connected to MongoDB!")
except Exception as e:
    print(e)

Pinged your deployment. You successfully connected to MongoDB!


In [3]:
# create the blog table and insert a test document
# db = client.blog
# db.posts.insert_one({"title": "test post", "body": "this is a test post"})
# print("Inserted a test document into the posts collection.")

Inserted a test document into the posts collection.


In [11]:
# now given a jupyter notebook, we can convert it to an html file and then
# upload that file to the blog database
def convert_and_upload(notebook, theme):
    import nbconvert
    import nbformat
    import os
    import datetime

    # convert the notebook to html
    html = nbconvert.HTMLExporter()
    html.theme = theme
    html.exclude_input = True
    html.exclude_output_prompt = True
    html.exclude_input_prompt = True
    html.exclude_output = True

    # get the name of the notebook
    name = notebook.split('.')[0]

    # read the notebook
    with open(notebook) as f:
        nb = nbformat.read(f, as_version=4)

    # convert it to html
    (body, resources) = html.from_notebook_node(nb)

    # create the html file
    with open(name + '.html', 'w') as f:
        f.write(body)

    # upload the html file to the blog database
    db = client.blog
    db.posts.insert_one({"title": name, "body": body, "date": datetime.datetime.now()})
    print("Inserted a test document into the posts collection.")

    # remove the html file
    os.remove(name + '.html')

convert_and_upload('test2.ipynb', 'dark')


Inserted a test document into the posts collection.


In [10]:
# pull the second document from the posts collection
post = db.posts.find()[1]
# save it as an html file
with open(post['title'] + '.html', 'w') as f:
    f.write(post['body'])